In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = "Pistachio_Image_Dataset"
BATCH_SIZE = 32
TARGET_ACC = 0.75

print("Device:", device)

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total



print("CNN")

transform_cnn = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(DATA_DIR, transform=transform_cnn)
num_classes = len(dataset.classes)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.classifier(self.features(x))

cnn = CNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=1e-3)

cnn_final_acc = 0
cnn_convergence_epoch = None

EPOCHS_CNN = 15

for epoch in range(EPOCHS_CNN):
    cnn.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(cnn(x), y)
        loss.backward()
        optimizer.step()

    val_acc = evaluate(cnn, val_loader)
    cnn_final_acc = val_acc

    if cnn_convergence_epoch is None and val_acc >= TARGET_ACC:
        cnn_convergence_epoch = epoch + 1

    print(f"[CNN] Epoch {epoch+1}: Accuracy = {val_acc:.4f}")



# Take RESNET18

print("RESNET18")

transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

dataset = datasets.ImageFolder(DATA_DIR, transform=transform_resnet)
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

for param in resnet.parameters():
    param.requires_grad = False

resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
resnet = resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=1e-4)

resnet_final_acc = 0
resnet_convergence_epoch = None

EPOCHS_RESNET = 8

for epoch in range(EPOCHS_RESNET):
    resnet.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(resnet(x), y)
        loss.backward()
        optimizer.step()

    val_acc = evaluate(resnet, val_loader)
    resnet_final_acc = val_acc

    if resnet_convergence_epoch is None and val_acc >= TARGET_ACC:
        resnet_convergence_epoch = epoch + 1

    print(f"[ResNet] Epoch {epoch+1}: Accuracy = {val_acc:.4f}")




print(f"Total dataset size: {len(dataset)} images")
print(f"Train size: {len(train_ds)}, Validation size: {len(val_ds)}\n")

print("CNN:")
print(f"  Accuracy: {cnn_final_acc:.4f}")

print("ResNet18:")
print(f"  Accuracy: {resnet_final_acc:.4f}")

Device: cpu
CNN
[CNN] Epoch 1: Accuracy = 0.7837
[CNN] Epoch 2: Accuracy = 0.8023
[CNN] Epoch 3: Accuracy = 0.8070
[CNN] Epoch 4: Accuracy = 0.8395
[CNN] Epoch 5: Accuracy = 0.8535
[CNN] Epoch 6: Accuracy = 0.8535
[CNN] Epoch 7: Accuracy = 0.8372
[CNN] Epoch 8: Accuracy = 0.8581
[CNN] Epoch 9: Accuracy = 0.8488
[CNN] Epoch 10: Accuracy = 0.8419
[CNN] Epoch 11: Accuracy = 0.8488
[CNN] Epoch 12: Accuracy = 0.8651
[CNN] Epoch 13: Accuracy = 0.8419
[CNN] Epoch 14: Accuracy = 0.8721
[CNN] Epoch 15: Accuracy = 0.8744
RESNET18
[ResNet] Epoch 1: Accuracy = 0.6930
[ResNet] Epoch 2: Accuracy = 0.7651
[ResNet] Epoch 3: Accuracy = 0.8465
[ResNet] Epoch 4: Accuracy = 0.8791
[ResNet] Epoch 5: Accuracy = 0.8860
[ResNet] Epoch 6: Accuracy = 0.9070
[ResNet] Epoch 7: Accuracy = 0.8977
[ResNet] Epoch 8: Accuracy = 0.9093
Total dataset size: 2148 images
Train size: 1718, Validation size: 430

CNN:
  Accuracy: 0.8744
ResNet18:
  Accuracy: 0.9093
